In [1]:
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd

In [2]:
movies = pd.read_csv('../lecture-1/movies.csv')
ratings = pd.read_csv('../lecture-1/ratings.csv')

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [5]:
movies_with_ratings[movies_with_ratings.userId == 2.0].title.unique()

array(['Shawshank Redemption, The (1994)', 'Tommy Boy (1995)',
       'Good Will Hunting (1997)', 'Gladiator (2000)',
       'Kill Bill: Vol. 1 (2003)', 'Collateral (2004)',
       'Talladega Nights: The Ballad of Ricky Bobby (2006)',
       'Departed, The (2006)', 'Dark Knight, The (2008)',
       'Step Brothers (2008)', 'Inglourious Basterds (2009)',
       'Zombieland (2009)', 'Shutter Island (2010)',
       'Exit Through the Gift Shop (2010)', 'Inception (2010)',
       'Town, The (2010)', 'Inside Job (2010)',
       'Louis C.K.: Hilarious (2010)', 'Warrior (2011)',
       'Dark Knight Rises, The (2012)',
       'Girl with the Dragon Tattoo, The (2011)',
       'Django Unchained (2012)', 'Wolf of Wall Street, The (2013)',
       'Interstellar (2014)', 'Whiplash (2014)', 'The Drop (2014)',
       'Ex Machina (2015)', 'Mad Max: Fury Road (2015)',
       'The Jinx: The Life and Deaths of Robert Durst (2015)'],
      dtype=object)

In [6]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [7]:
dataset.head()

,uid,iid,rating
0,1.0,Toy Story (1995),4.0
1,5.0,Toy Story (1995),4.0
2,7.0,Toy Story (1995),4.5
3,15.0,Toy Story (1995),2.5
4,17.0,Toy Story (1995),4.5


In [8]:
ratings.rating.min()

0.5

In [9]:
ratings.rating.max()

5.0

In [10]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [11]:
trainset, testset = train_test_split(data, test_size=.15)

In [12]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [13]:
test_pred = algo.test(testset)

In [14]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8925


0.8924753503608442

In [15]:
predict = algo.predict(uid=2, iid='Fight Club (1999)')

In [16]:
predict

Prediction(uid=2, iid='Fight Club (1999)', r_ui=None, est=4.642326159001888, details={'actual_k': 50, 'was_impossible': False})

In [17]:
def analyze_users(uid, iid):
    n = algo.get_neighbors(uid,10)
    wighted = []
    print(n)
    for p in n:
        rating = movies_with_ratings[(movies_with_ratings['userId'] == p) & (movies_with_ratings['title'] == iid)]
        
        if len(rating.values) > 0:
            value = rating.rating.values[0]
            print(p, value, algo.sim[uid,p], algo.sim[uid,p] * value)
            wighted.append((algo.sim[uid,p] * value,algo.sim[uid,p]))
    return wighted
            
    

In [18]:
w = analyze_users(2, 'Fight Club (1999)')

[166, 222, 237, 608, 103, 74, 523, 341, 399, 275]
166 4.5 0.06689862389597584 0.3010438075318913
222 3.5 0.05918916279195915 0.20716206977185703
608 5.0 0.04599324517064704 0.2299662258532352
103 5.0 0.04403181950433818 0.2201590975216909
523 4.5 0.04027870735001999 0.18125418307508995
399 3.0 0.037438773051128506 0.11231631915338552
275 5.0 0.03697284647022765 0.18486423235113825


# HM

### SVD & GridSearch

In [19]:
from surprise.model_selection import GridSearchCV
from surprise import SVD

In [20]:
param_grid = {'n_epochs': [ 20,23], 'lr_all': [ 0.01,0.015,0.02],
              'reg_all': [  0.1, 0.15, 0.2]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

0.8605520153771821
{'n_epochs': 23, 'lr_all': 0.02, 'reg_all': 0.1}


### KNN & GridSearch

In [21]:
from surprise import KNNWithMeans, KNNBasic, KNNBaseline, KNNWithZScore
import numpy as np

In [22]:
benchmark = []
# Iterate over all algorithms
for algorithm in [ KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore]:
    param_grid = {
              'k': [40,50,55,35],
              'sim_options': {'name': ['msd', 'cosine'],
                              'min_support': [10,20,30],
                              'user_based': [True]}
              }
    gs = GridSearchCV(algorithm, param_grid, measures=['rmse', 'mae'], cv=3)

    gs.fit(data)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(gs.cv_results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[1].split('.')[-1].replace(">",'')], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')   

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity m

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd simila

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity m

KeyError: 'test_rmse'

In [23]:
pd.DataFrame(benchmark).set_index('Algorithm')

,split0_test_rmse,split1_test_rmse,split2_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,split2_test_mae,mean_test_mae,std_test_mae,rank_test_mae,mean_fit_time,std_fit_time,mean_test_time,std_test_time,param_k
Algorithm,,,,,,,,,,,,,,,,,
KNNBaseline',0.892311,0.898990,0.897719,0.896340,0.003158,12.5,0.680755,0.687289,0.686696,0.684913,0.003093,12.5,0.350590,0.012731,3.398550,0.065817,45.0
KNNBasic',0.994700,0.986632,0.992838,0.991390,0.003508,12.5,0.769730,0.764760,0.767867,0.767453,0.002088,12.5,0.151830,0.008309,2.642052,0.059949,45.0
KNNWithMeans',0.927090,0.921417,0.922121,0.923543,0.002782,12.5,0.705837,0.700270,0.704461,0.703523,0.002386,12.5,0.185119,0.007577,2.791424,0.068240,45.0
KNNWithZScore',0.927383,0.917080,0.921593,0.922019,0.004264,12.5,0.698243,0.693575,0.697108,0.696309,0.002013,12.5,0.256189,0.007461,3.047552,0.047114,45.0


In [ ]:

param_grid = {'k': [ 40, 45, 50],
              'sim_options': {'name': ['pearson_baseline','pearson_baseline','msd'],'min_support':[3,5],
                              'user_based': [False,True]
                             },
              'bsl_options': {'method': ['als','sgd'],
                              'reg_i': [8,10,12],
                              'reg_u': [3,8,10,15,17]
                             }
             }

gs = GridSearchCV(KNNBaseline, param_grid, measures=['rmse'], cv=3, return_train_measures=True)
gs.fit(data)

In [24]:

param_grid = {'k': [ 45, 50],
              'sim_options': {'name': ['pearson_baseline','msd'],'min_support':[3,5],
                              'user_based': [True]
                             },
              'bsl_options': {'method': ['als'],
                              'reg_i': [8,10],
                              'reg_u': [0,15]
                             }
             }

gs = GridSearchCV(KNNBaseline, param_grid, measures=['rmse'], cv=3, return_train_measures=True)
gs.fit(data)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline si

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matr

In [25]:
gs.cv_results

{'split0_test_rmse': array([0.88814089, 0.88930779, 0.88747968, 0.88840516, 0.88988539,
        0.89111937, 0.88933981, 0.89030896, 0.88002338, 0.88053701,
        0.8805092 , 0.88090427, 0.88136435, 0.88230945, 0.88186486,
        0.88268958, 0.8881564 , 0.88933406, 0.8874873 , 0.88842616,
        0.88991715, 0.89113754, 0.88936576, 0.89032336, 0.88011284,
        0.88066266, 0.88059576, 0.88102823, 0.88155593, 0.88251712,
        0.88205376, 0.88289565]),
 'split0_train_rmse': array([0.24184531, 0.24092823, 0.24836521, 0.24733941, 0.2410822 ,
        0.24022269, 0.24763316, 0.24665047, 0.66785959, 0.66661741,
        0.66858927, 0.66720017, 0.66969305, 0.66894467, 0.67041663,
        0.66952371, 0.24243043, 0.24148067, 0.24895329, 0.24790121,
        0.24154009, 0.24064769, 0.24809206, 0.24710477, 0.67095697,
        0.66988281, 0.67168349, 0.67046396, 0.67242992, 0.6718083 ,
        0.67315111, 0.67238616]),
 'split1_test_rmse': array([0.88956883, 0.89089117, 0.88891414, 0.89022591,

In [26]:
tmp = pd.DataFrame.from_dict(gs.cv_results).mean(axis=0)
tmp = tmp.append(pd.Series([str(algorithm).split(' ')[1].split('.')[-1].replace(">",'')], index=['Algorithm']))
benchmark.append(tmp)

In [27]:
pd.DataFrame(benchmark).set_index('Algorithm')

,split0_test_rmse,split1_test_rmse,split2_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,split2_test_mae,mean_test_mae,...,mean_fit_time,std_fit_time,mean_test_time,std_test_time,param_k,split0_train_rmse,split1_train_rmse,split2_train_rmse,mean_train_rmse,std_train_rmse
Algorithm,,,,,,,,,,,,,,,,,,,,,
KNNBaseline',0.892311,0.898990,0.897719,0.896340,0.003158,12.5,0.680755,0.687289,0.686696,0.684913,...,0.350590,0.012731,3.398550,0.065817,45.0,NaN,NaN,NaN,NaN,NaN
KNNBasic',0.994700,0.986632,0.992838,0.991390,0.003508,12.5,0.769730,0.764760,0.767867,0.767453,...,0.151830,0.008309,2.642052,0.059949,45.0,NaN,NaN,NaN,NaN,NaN
KNNWithMeans',0.927090,0.921417,0.922121,0.923543,0.002782,12.5,0.705837,0.700270,0.704461,0.703523,...,0.185119,0.007577,2.791424,0.068240,45.0,NaN,NaN,NaN,NaN,NaN
KNNWithZScore',0.927383,0.917080,0.921593,0.922019,0.004264,12.5,0.698243,0.693575,0.697108,0.696309,...,0.256189,0.007461,3.047552,0.047114,45.0,NaN,NaN,NaN,NaN,NaN
KNNWithZScore',0.885305,0.885263,0.895022,0.888530,0.004626,16.5,NaN,NaN,NaN,NaN,...,0.412432,0.011993,3.727003,0.060145,47.5,0.457307,0.456285,0.455584,0.456392,0.00165
